In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from scipy.stats import pearsonr
from collections import Counter

# importamos as libs
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
import numpy as np
import tensorflow as tf
sns.set_palette("Set2")

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
# calcula metricas de regressao
def score_regression_metrics(y_test, y_test_pred):

    RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "neg_root_mean_squared_error": RMSE,
        "neg_mean_absolute_error": MAE,
        "neg_mean_absolute_percentage_error": MAPE,
    }

    return scores

def show_scores (scores):

    print ("Scores obtidos:")

    print (f"RMSE: {scores['neg_root_mean_squared_error']}")
    print (f"MAE: {scores['neg_mean_absolute_error']}")
    print (f"MAPE: {scores['neg_mean_absolute_percentage_error']}")

In [3]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [4]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [5]:
def set_seeds():
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

In [216]:
df = pd.read_csv("input_and_output.csv")

In [217]:
SEED=47

TO_DROP = [
    "pib_pc",
    "idh_long",
    "nfsp",
    "estoque",
]

VARS = [
    'pib_pmc',
    'pib_pcpt',
    'populacao',
    'pib_cc',
    'ipca',
    'incc',
    'igp',
    'selic',
    'idh_renda',
    'idh_educacao',
    'desemprego'
]

batch_size = 32
epochs = 200
verbose = 1
timesteps = 3

In [218]:
# TO_DROP = [

# ]

# VARS = [
#     "pib_pc",
#     "idh_long",
#     "nfsp",
#     "estoque",
#     'pib_pmc',
#     'pib_pcpt',
#     'populacao',
#     'pib_cc',
#     'ipca',
#     'incc',
#     'igp',
#     'selic',
#     'idh_renda',
#     'idh_educacao',
#     'desemprego'
# ]

In [219]:
df = df.drop(TO_DROP, axis=1)

In [220]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [221]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [222]:
# df[VARS].hist(bins=25, figsize=(20, 20), layout=(-1, 5), edgecolor="black")
# plt.tight_layout()

In [223]:
df_sorted = df.sort_values(by=["estados", "data"])
for estado in df.estados.unique():
    indexes = df_sorted[df_sorted["estados"] == estado].index
    df_sorted.loc[indexes, "producao"] = df_sorted.loc[indexes].producao.shift(1)

In [224]:
df_sorted = df_sorted[df_sorted.producao.notna()]

In [240]:
df_sorted = df_sorted.sort_values(by=["data", "estados"])

In [241]:
df = df_sorted

In [210]:
x = df.drop(["data", "consumo"], axis=1).copy()

# processo de one-hot
x = pd.get_dummies(data=x, columns=["estados"], drop_first=True)

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [211]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.9611790983876279


In [212]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 40425.21197054409
MAE: 24012.83146913503
MAPE: 0.46357947133600264

TESTE
Scores obtidos:
RMSE: 35298.35252173542
MAE: 24843.3921823251
MAPE: 0.4208661889241405


In [234]:
x = df.drop(["data", "consumo"], axis=1).copy()

# processo de one-hot
x = pd.get_dummies(data=x, columns=["estados"], drop_first=True)

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [235]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.8169891627286769


In [236]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 32378.898120001966
MAE: 20618.38722036052
MAPE: 0.44035108655585387

TESTE
Scores obtidos:
RMSE: 177479.21877353307
MAE: 108770.17895378474
MAPE: 0.4204635278380479


In [237]:
x = df.drop(["data", "estados", "consumo"], axis=1).copy()

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [238]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.9549585423787639


In [239]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 51061.349763018276
MAE: 32024.049617912762
MAPE: 0.651155608095214

TESTE
Scores obtidos:
RMSE: 88047.15402996687
MAE: 54282.28229742509
MAPE: 0.34291039958174513


In [181]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=256, activation="relu")),
    model.add(tf.keras.layers.Dense(units=128, activation="relu")),
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [242]:
x = df.drop(["estados", "data", "consumo", "producao"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 2ms/step - loss: 68131569664.0000 - RMSE: 261020.2500
Epoch 2/500
146/146 [==============================] - 0s 2ms/step - loss: 17626572800.0000 - RMSE: 132765.1094
Epoch 3/500
146/146 [==============================] - 0s 2ms/step - loss: 7076975616.0000 - RMSE: 84124.7656
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 5907714048.0000 - RMSE: 76861.6562
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 5320099840.0000 - RMSE: 72939.0156
Epoch 6/500
146/146 [==============================] - 0s 2ms/step - loss: 5064832512.0000 - RMSE: 71167.6328
Epoch 7/500
146/146 [==============================] - 0s 3ms/step - loss: 4793539072.0000 - RMSE: 69235.3906
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 4359118848.0000 - RMSE: 66023.6250
Epoch 9/500
146/146 [==============================] - 0s 2ms/step - loss: 4387690496.0000 - RMSE: 66239.6406
Epoch 

In [243]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 26303.13542026353
MAE: 15853.766910153734
MAPE: 0.24144434085420163
TESTE
Scores obtidos:
RMSE: 45423.81502263361
MAE: 31136.91863264638
MAPE: 0.5355255019178464


In [ ]:
================
TRAIN
================
Scores obtidos:
RMSE: 22807.097324172653
MAE: 13047.578577826669
MAPE: 0.15626501595083667
================
TESTE
================
Scores obtidos:
RMSE: 38398.31929394046
MAE: 25176.616115288693
MAPE: 0.34854061088685906

In [245]:
x = df.drop(["estados", "data", "consumo"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 67369742336.0000 - RMSE: 259556.8125
Epoch 2/500
146/146 [==============================] - 0s 1ms/step - loss: 18777323520.0000 - RMSE: 137030.3750
Epoch 3/500
146/146 [==============================] - 0s 1ms/step - loss: 7207237120.0000 - RMSE: 84895.4453
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 6077259776.0000 - RMSE: 77956.7812
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 5505097216.0000 - RMSE: 74196.3438
Epoch 6/500
146/146 [==============================] - 0s 1ms/step - loss: 5358016512.0000 - RMSE: 73198.4766
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 4675815936.0000 - RMSE: 68379.9375
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 4355995648.0000 - RMSE: 65999.9688
Epoch 9/500
146/146 [==============================] - 0s 2ms/step - loss: 3921143040.0000 - RMSE: 62619.0312
Epoch 

In [246]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 26075.572447140705
MAE: 16196.812356806335
MAPE: 0.2907686991361337
TESTE
Scores obtidos:
RMSE: 27551.9014990335
MAE: 18407.06254399871
MAPE: 0.39691104317960985


In [248]:
x = pd.get_dummies(data=df, columns=["estados"], drop_first=True).drop(["data", "consumo"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 65195859968.0000 - RMSE: 255334.7969
Epoch 2/500
146/146 [==============================] - 0s 2ms/step - loss: 8525268480.0000 - RMSE: 92332.3828
Epoch 3/500
146/146 [==============================] - 0s 3ms/step - loss: 3250401792.0000 - RMSE: 57012.2969
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 2729658624.0000 - RMSE: 52246.1367
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 2515936256.0000 - RMSE: 50159.1094
Epoch 6/500
146/146 [==============================] - 0s 2ms/step - loss: 2611849728.0000 - RMSE: 51106.2578
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 2251320832.0000 - RMSE: 47448.0859
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 2081638528.0000 - RMSE: 45624.9766
Epoch 9/500
146/146 [==============================] - 0s 3ms/step - loss: 1940155264.0000 - RMSE: 44047.1953
Epoch 10

In [249]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 22118.94515508848
MAE: 11347.287327778968
MAPE: 0.10623667394327253
TESTE
Scores obtidos:
RMSE: 25073.523052175206
MAE: 14999.138322926556
MAPE: 0.17874258235578142


In [251]:
x = pd.get_dummies(data=df, columns=["estados"], drop_first=True).drop(["data", "consumo", "producao"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 65459220480.0000 - RMSE: 255850.0000
Epoch 2/500
146/146 [==============================] - 0s 1ms/step - loss: 8927627264.0000 - RMSE: 94486.1250
Epoch 3/500
146/146 [==============================] - 0s 2ms/step - loss: 3292027648.0000 - RMSE: 57376.1953
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 2770957312.0000 - RMSE: 52639.8828
Epoch 5/500
146/146 [==============================] - 0s 1ms/step - loss: 2537138176.0000 - RMSE: 50370.0117
Epoch 6/500
146/146 [==============================] - 0s 1ms/step - loss: 2642412288.0000 - RMSE: 51404.3984
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 2251692800.0000 - RMSE: 47452.0039
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 2097421056.0000 - RMSE: 45797.6094
Epoch 9/500
146/146 [==============================] - 0s 3ms/step - loss: 1952263040.0000 - RMSE: 44184.4219
Epoch 10

In [252]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 22842.974044750586
MAE: 11991.906759703725
MAPE: 0.12137462404970259
TESTE
Scores obtidos:
RMSE: 24638.211950348432
MAE: 14591.643175961613
MAPE: 0.18156455930319815


In [266]:
df_copy = df[(df["data"] >= 200402) & (df["data"] <= 200412)].copy()
x = pd.get_dummies(data=df_copy, columns=["estados"], drop_first=True)
train_filter = (x["data"] < 200412)
x_train = x[train_filter].drop(["data", "consumo"], axis=1)
y_train = x[train_filter]["consumo"]

x_test = x[~train_filter].drop(["data", "consumo"], axis=1)
y_test = x[~train_filter]["consumo"]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
9/9 [==============================] - 0s 2ms/step - loss: 34233198592.0000 - RMSE: 185022.1562
Epoch 2/500
9/9 [==============================] - 0s 4ms/step - loss: 34232410112.0000 - RMSE: 185020.0312
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 34230390784.0000 - RMSE: 185014.5625
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 34225027072.0000 - RMSE: 185000.0781
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 34210074624.0000 - RMSE: 184959.6562
Epoch 6/500
9/9 [==============================] - 0s 3ms/step - loss: 34181173248.0000 - RMSE: 184881.5156
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 34119145472.0000 - RMSE: 184713.6875
Epoch 8/500
9/9 [==============================] - 0s 3ms/step - loss: 33995280384.0000 - RMSE: 184378.0938
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 33740806144.0000 - RMSE: 183686.7031
Epoch 10/500
9/9 [==========

In [267]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 11365.361984920366
MAE: 5737.867214098668
MAPE: 0.0920544299785111
TESTE
Scores obtidos:
RMSE: 20251.455116795565
MAE: 16771.309583912036
MAPE: 0.6147865217346026


In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=32, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [ ]:
x, y = split_sequences(df.drop(["estados", "data", "consumo"], axis=1).values, 15)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=128, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [ ]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, timesteps)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

In [ ]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, 7)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)